# Guided Project : Linear Regression

We will practice what we learned in our previous lessons to build a ML model here. We will be working with the housing data for the city of Ames, Iowa from 2006 to 2010.



In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import math

housing = pd.read_csv('AmesHousing.tsv',delimiter = '\t')

In [2]:
housing.shape

(2930, 82)

In [3]:
train = housing[:int(2930/2)]
test = housing[int(2390/2):]

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 82 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Order            1465 non-null   int64  
 1   PID              1465 non-null   int64  
 2   MS SubClass      1465 non-null   int64  
 3   MS Zoning        1465 non-null   object 
 4   Lot Frontage     1213 non-null   float64
 5   Lot Area         1465 non-null   int64  
 6   Street           1465 non-null   object 
 7   Alley            109 non-null    object 
 8   Lot Shape        1465 non-null   object 
 9   Land Contour     1465 non-null   object 
 10  Utilities        1465 non-null   object 
 11  Lot Config       1465 non-null   object 
 12  Land Slope       1465 non-null   object 
 13  Neighborhood     1465 non-null   object 
 14  Condition 1      1465 non-null   object 
 15  Condition 2      1465 non-null   object 
 16  Bldg Type        1465 non-null   object 
 17  House Style   

As we can see, there are columns with missing values that will not be needed.

First of all we will solve the basic by just the deleting the columns on the basis of proportion of missing values.

In [5]:
# we will delete the columns with more than 10% missing values
null_cols = train.isnull().sum()
missing_cols = null_cols[null_cols >15]
train = train.drop(missing_cols.index, axis = 1)

In [6]:
train

,Order,PID,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,...,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,31770,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,11622,Pave,Reg,Lvl,AllPub,Inside,...,0,0,120,0,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,14267,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,11160,Pave,Reg,Lvl,AllPub,Corner,...,0,0,0,0,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,13830,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,3,2010,WD,Normal,189900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,1461,907255030,60,RL,9720,Pave,IR1,Lvl,AllPub,CulDSac,...,0,0,0,0,0,1,2008,WD,Normal,274000
1461,1462,907255060,20,RL,14860,Pave,IR2,Lvl,AllPub,CulDSac,...,0,0,0,0,0,6,2008,WD,Normal,300000
1462,1463,907260010,60,RL,11250,Pave,Reg,Lvl,AllPub,Corner,...,0,0,0,0,0,5,2008,WD,Normal,255900
1463,1464,907262030,60,RL,8158,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,7,2008,WD,Normal,224900


In [7]:
# now the text columns with any missing value will be dropped
text_mv_counts = train.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)

## Filter Series to columns containing *any* missing values
drop_missing_cols_2 = text_mv_counts[text_mv_counts > 0]

train = train.drop(drop_missing_cols_2.index, axis=1)

In [8]:
train

,Order,PID,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,...,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,31770,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,11622,Pave,Reg,Lvl,AllPub,Inside,...,0,0,120,0,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,14267,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,11160,Pave,Reg,Lvl,AllPub,Corner,...,0,0,0,0,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,13830,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,3,2010,WD,Normal,189900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,1461,907255030,60,RL,9720,Pave,IR1,Lvl,AllPub,CulDSac,...,0,0,0,0,0,1,2008,WD,Normal,274000
1461,1462,907255060,20,RL,14860,Pave,IR2,Lvl,AllPub,CulDSac,...,0,0,0,0,0,6,2008,WD,Normal,300000
1462,1463,907260010,60,RL,11250,Pave,Reg,Lvl,AllPub,Corner,...,0,0,0,0,0,5,2008,WD,Normal,255900
1463,1464,907262030,60,RL,8158,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,7,2008,WD,Normal,224900


In [9]:
# for numerical columns we will fill in with the most commmon value
num_missing = train.select_dtypes(include=['int', 'float']).isnull().sum()
fixable_numeric_cols = num_missing[(num_missing < len(train)/20) & (num_missing > 0)].sort_values()
fixable_numeric_cols

BsmtFin SF 1       1
BsmtFin SF 2       1
Bsmt Unf SF        1
Total Bsmt SF      1
Bsmt Full Bath     1
Bsmt Half Bath     1
Mas Vnr Area      11
dtype: int64

In [29]:
replacement_values_dict = train[fixable_numeric_cols.index].mode().to_dict(orient='records')[0]
## Use `pd.DataFrame.fillna()` to replace missing values.
train = train.fillna(replacement_values_dict)
replacement_values_dict

{'BsmtFin SF 1': 0.0,
 'BsmtFin SF 2': 0.0,
 'Bsmt Unf SF': 0.0,
 'Total Bsmt SF': 864.0,
 'Bsmt Full Bath': 0.0,
 'Bsmt Half Bath': 0.0,
 'Mas Vnr Area': 0.0}

In [11]:
train.isnull().sum()

Order             0
PID               0
MS SubClass       0
MS Zoning         0
Lot Area          0
                 ..
Mo Sold           0
Yr Sold           0
Sale Type         0
Sale Condition    0
SalePrice         0
Length: 65, dtype: int64

In [12]:
# Now we would want to create new features
# Years in which house was sold after it was built

years_sold = train['Yr Sold'] - train['Year Built']
years_since_remod = train['Yr Sold'] - train['Year Remod/Add']

In [13]:
# we will try to catch errors now. We will eleimnate the rows which claim years_sold or years_since_remod
# is negative as it cant be.

print(years_sold[years_sold < 0])
print(years_since_remod[years_since_remod < 0])

Series([], dtype: int64)
Series([], dtype: int64)


In [14]:
train.drop(['Year Built','Yr Sold'],axis = 1)

,Order,PID,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,...,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,31770,Pave,IR1,Lvl,AllPub,Corner,...,62,0,0,0,0,0,5,WD,Normal,215000
1,2,526350040,20,RH,11622,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,120,0,0,6,WD,Normal,105000
2,3,526351010,20,RL,14267,Pave,IR1,Lvl,AllPub,Corner,...,36,0,0,0,0,12500,6,WD,Normal,172000
3,4,526353030,20,RL,11160,Pave,Reg,Lvl,AllPub,Corner,...,0,0,0,0,0,0,4,WD,Normal,244000
4,5,527105010,60,RL,13830,Pave,IR1,Lvl,AllPub,Inside,...,34,0,0,0,0,0,3,WD,Normal,189900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,1461,907255030,60,RL,9720,Pave,IR1,Lvl,AllPub,CulDSac,...,116,0,0,0,0,0,1,WD,Normal,274000
1461,1462,907255060,20,RL,14860,Pave,IR2,Lvl,AllPub,CulDSac,...,35,0,0,0,0,0,6,WD,Normal,300000
1462,1463,907260010,60,RL,11250,Pave,Reg,Lvl,AllPub,Corner,...,0,0,0,0,0,0,5,WD,Normal,255900
1463,1464,907262030,60,RL,8158,Pave,Reg,Lvl,AllPub,Inside,...,63,0,0,0,0,0,7,WD,Normal,224900


In [15]:
# As done before practice we will drop columns which are not useful for our model
useless_columns = ['PID','Order',"Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"]
train = train.drop(useless_columns, axis = 1)

In [16]:
train.head()

,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,...,Garage Area,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,SalePrice
0,20,RL,31770,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,...,528.0,P,210,62,0,0,0,0,0,215000
1,20,RH,11622,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,...,730.0,Y,140,0,0,0,120,0,0,105000
2,20,RL,14267,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,...,312.0,Y,393,36,0,0,0,0,12500,172000
3,20,RL,11160,Pave,Reg,Lvl,AllPub,Corner,Gtl,NAmes,...,522.0,Y,0,0,0,0,0,0,0,244000
4,60,RL,13830,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,...,482.0,Y,212,34,0,0,0,0,0,189900


## Feature Selection


In [17]:
# sort out the numerical columns and select the ones with correlation with SalePrice more than 0.4
numerical = train.select_dtypes(include = ['int','float'])
numerical.head()

,MS SubClass,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,...,Garage Cars,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,SalePrice
0,20,31770,6,5,1960,1960,112.0,639.0,0.0,441.0,...,2.0,528.0,210,62,0,0,0,0,0,215000
1,20,11622,5,6,1961,1961,0.0,468.0,144.0,270.0,...,1.0,730.0,140,0,0,0,120,0,0,105000
2,20,14267,6,6,1958,1958,108.0,923.0,0.0,406.0,...,1.0,312.0,393,36,0,0,0,0,12500,172000
3,20,11160,7,5,1968,1968,0.0,1065.0,0.0,1045.0,...,2.0,522.0,0,0,0,0,0,0,0,244000
4,60,13830,5,5,1997,1998,0.0,791.0,0.0,137.0,...,2.0,482.0,212,34,0,0,0,0,0,189900


In [18]:
#this is the correlation that matters to us
corr = numerical.corr()['SalePrice']
corr = abs(corr).sort_values()

In [19]:
# these will be our final features based on correlation
final_corr = corr[corr >= 0.4] 
final_corr

TotRms AbvGrd     0.484053
Fireplaces        0.485892
Mas Vnr Area      0.513052
BsmtFin SF 1      0.513512
Full Bath         0.519375
Year Remod/Add    0.546442
Year Built        0.592749
1st Flr SF        0.656942
Garage Area       0.663488
Garage Cars       0.664468
Total Bsmt SF     0.683265
Gr Liv Area       0.699537
Overall Qual      0.805201
SalePrice         1.000000
Name: SalePrice, dtype: float64

In [20]:
transform_df = train[final_corr.index]
transform_df.head()

,TotRms AbvGrd,Fireplaces,Mas Vnr Area,BsmtFin SF 1,Full Bath,Year Remod/Add,Year Built,1st Flr SF,Garage Area,Garage Cars,Total Bsmt SF,Gr Liv Area,Overall Qual,SalePrice
0,7,2,112.0,639.0,1,1960,1960,1656,528.0,2.0,1080.0,1656,6,215000
1,5,0,0.0,468.0,1,1961,1961,896,730.0,1.0,882.0,896,5,105000
2,6,0,108.0,923.0,1,1958,1958,1329,312.0,1.0,1329.0,1329,6,172000
3,8,2,0.0,1065.0,2,1968,1968,2110,522.0,2.0,2110.0,2110,7,244000
4,6,1,0.0,791.0,2,1998,1997,928,482.0,2.0,928.0,1629,5,189900


In [21]:
## Create a list of column names from documentation that are *meant* to be categorical
nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

In [22]:
## Which categorical columns have we still carried with us? We'll test these 
transform_cat_cols = []
for col in nominal_features:
    if col in transform_df.columns:
        transform_cat_cols.append(col)

## How many unique values in each categorical column?
uniqueness_counts = transform_df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
## Aribtrary cutoff of 10 unique values (worth experimenting)
drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index
transform_df = transform_df.drop(drop_nonuniq_cols, axis=1)
transform_df

,TotRms AbvGrd,Fireplaces,Mas Vnr Area,BsmtFin SF 1,Full Bath,Year Remod/Add,Year Built,1st Flr SF,Garage Area,Garage Cars,Total Bsmt SF,Gr Liv Area,Overall Qual,SalePrice
0,7,2,112.0,639.0,1,1960,1960,1656,528.0,2.0,1080.0,1656,6,215000
1,5,0,0.0,468.0,1,1961,1961,896,730.0,1.0,882.0,896,5,105000
2,6,0,108.0,923.0,1,1958,1958,1329,312.0,1.0,1329.0,1329,6,172000
3,8,2,0.0,1065.0,2,1968,1968,2110,522.0,2.0,2110.0,2110,7,244000
4,6,1,0.0,791.0,2,1998,1997,928,482.0,2.0,928.0,1629,5,189900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,7,1,134.0,1194.0,2,2002,2001,1366,725.0,3.0,1357.0,1947,9,274000
1461,6,1,240.0,1538.0,2,2003,2002,1786,715.0,3.0,1778.0,1786,8,300000
1462,9,1,0.0,0.0,2,2002,2002,1149,779.0,2.0,1128.0,2290,8,255900
1463,8,0,252.0,550.0,2,2003,2003,884,543.0,2.0,884.0,1768,7,224900


In [23]:
text_cols = transform_df.select_dtypes(include=['object'])
for col in text_cols:
    transform_df[col] = transform_df[col].astype('category')
'''## Create dummy columns and add back to the dataframe!
transform_df = pd.concat([
    transform_df, 
    pd.get_dummies(transform_df.select_dtypes(include=['category']))
], axis=1).drop(text_cols,axis=1)'''
for col in text_cols:
    col_dummies = pd.get_dummies(transform_df[col])
    transform_df = pd.concat([transform_df, col_dummies], axis=1)
    del transform_df[col]

In [24]:
transform_df

,TotRms AbvGrd,Fireplaces,Mas Vnr Area,BsmtFin SF 1,Full Bath,Year Remod/Add,Year Built,1st Flr SF,Garage Area,Garage Cars,Total Bsmt SF,Gr Liv Area,Overall Qual,SalePrice
0,7,2,112.0,639.0,1,1960,1960,1656,528.0,2.0,1080.0,1656,6,215000
1,5,0,0.0,468.0,1,1961,1961,896,730.0,1.0,882.0,896,5,105000
2,6,0,108.0,923.0,1,1958,1958,1329,312.0,1.0,1329.0,1329,6,172000
3,8,2,0.0,1065.0,2,1968,1968,2110,522.0,2.0,2110.0,2110,7,244000
4,6,1,0.0,791.0,2,1998,1997,928,482.0,2.0,928.0,1629,5,189900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,7,1,134.0,1194.0,2,2002,2001,1366,725.0,3.0,1357.0,1947,9,274000
1461,6,1,240.0,1538.0,2,2003,2002,1786,715.0,3.0,1778.0,1786,8,300000
1462,9,1,0.0,0.0,2,2002,2002,1149,779.0,2.0,1128.0,2290,8,255900
1463,8,0,252.0,550.0,2,2003,2003,884,543.0,2.0,884.0,1768,7,224900


In [30]:
# to summarise what we did above
import pandas as pd
pd.options.display.max_columns = 999
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.model_selection import KFold
def transform_features(df):
    num_missing = df.isnull().sum()
    drop_missing_cols = num_missing[(num_missing > len(df)/20)].sort_values()
    df = df.drop(drop_missing_cols.index, axis=1)
    
    text_mv_counts = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
    drop_missing_cols_2 = text_mv_counts[text_mv_counts > 0]
    df = df.drop(drop_missing_cols_2.index, axis=1)
    
    num_missing = df.select_dtypes(include=['int', 'float']).isnull().sum()
    fixable_numeric_cols = num_missing[(num_missing < len(df)/20) & (num_missing > 0)].sort_values()
    replacement_values_dict = df[fixable_numeric_cols.index].mode().to_dict(orient='records')[0]
    df = df.fillna(replacement_values_dict)
    
    years_sold = df['Yr Sold'] - df['Year Built']
    years_since_remod = df['Yr Sold'] - df['Year Remod/Add']
    df['Years Before Sale'] = years_sold
    df['Years Since Remod'] = years_since_remod
    df = df.drop([1702, 2180, 2181], axis=0)

    df = df.drop(["PID", "Order", "Mo Sold", "Sale Condition", "Sale Type", "Year Built", "Year Remod/Add"], axis=1)
    return df

def select_features(df, coeff_threshold=0.4, uniq_threshold=10):
    numerical_df = df.select_dtypes(include=['int', 'float'])
    abs_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values()
    df = df.drop(abs_corr_coeffs[abs_corr_coeffs < coeff_threshold].index, axis=1)
    
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]
    
    transform_cat_cols = []
    for col in nominal_features:
        if col in df.columns:
            transform_cat_cols.append(col)

    uniqueness_counts = df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
    drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index
    df = df.drop(drop_nonuniq_cols, axis=1)
    
    text_cols = df.select_dtypes(include=['object'])
    for col in text_cols:
        df[col] = df[col].astype('category')
    print(df.shape)
    df = pd.concat([df, pd.get_dummies(df.select_dtypes(include=['category']))], axis=1).drop(text_cols,axis=1)
    print(df.shape)
    return df

def train_and_test(df, k=0):
    numeric_df = df.select_dtypes(include=['integer', 'float'])
    features = numeric_df.columns.drop("SalePrice")
    lr = LinearRegression()
    
    if k == 0:
        train = df[:1460]
        test = df[1460:]

        lr.fit(train[features], train["SalePrice"])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(test["SalePrice"], predictions)
        rmse = np.sqrt(mse)

        return rmse
    
    if k == 1:
        # Randomize *all* rows (frac=1) from `df` and return
        shuffled_df = df.sample(frac=1, )
        train = df[:1460]
        test = df[1460:]
        
        lr.fit(train[features], train["SalePrice"])
        predictions_one = lr.predict(test[features])        
        
        mse_one = mean_squared_error(test["SalePrice"], predictions_one)
        rmse_one = np.sqrt(mse_one)
        
        lr.fit(test[features], test["SalePrice"])
        predictions_two = lr.predict(train[features])        
       
        mse_two = mean_squared_error(train["SalePrice"], predictions_two)
        rmse_two = np.sqrt(mse_two)
        
        avg_rmse = np.mean([rmse_one, rmse_two])
        print(rmse_one)
        print(rmse_two)
        return avg_rmse
    else:
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        for train_index, test_index, in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features], train["SalePrice"])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test["SalePrice"], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return avg_rmse

df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df, k=4)

rmse

(2927, 36)
(2927, 130)
[36284.09719627687, 29119.6086708909, 23288.190252332428, 27057.446232305494]


28937.33558795142